In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import warnings
np.random.seed(1234)
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

Let's try to see how well this approach works for our dataset. I would also like to point out that all these ideas and code are available at fast.ai's free official course for NLP - "".

In [ ]:
train_dataset = pd.read_csv("/kaggle/input/independence-hacakathon/train.csv", delimiter=",")
train_dataset

In [ ]:
test_dataset = pd.read_csv("/kaggle/input/independence-hacakathon/test.csv", delimiter=",")
test_dataset

In [ ]:
dataset = train_dataset.copy()
dataset

In [ ]:
dataset['Review_Combined'] = dataset['TITLE'] + ". " + dataset['ABSTRACT']
dataset['Review_Combined'][0]

In [ ]:
test_dataset['Review_Combined'] = test_dataset['TITLE'] + ". " + test_dataset['ABSTRACT']
test_dataset['Review_Combined'][0]

In [ ]:
test_dataset['ID']

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
#!pip install fastai2
from fastai.text import *

In [ ]:
from fastai import *

In [ ]:
from pathlib import Path

In [ ]:
dataset.to_csv("/kaggle/working/train.csv", index = False)

In [ ]:
path = Path('/kaggle/input/independence-hacakathon/')

Data in fast.ai is taken using TextLMDataBunch. This is very similar to ImageGenerator in Keras, where the path, labels, etc. are provided and the method prepares Train, Test and Validation data depending on the task at hand!

## Data Bunch for Language Model

In [ ]:
labels = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
#%%time
data_lm = TextLMDataBunch.from_csv(path,'train.csv', text_cols = "ABSTRACT", label_cols = labels)

## Data Bunch for Classification Task

In [ ]:
data_lm

In [ ]:
#%%time
data_clas = TextClasDataBunch.from_csv(path, csv_name = 'train.csv', test = 'test.csv', vocab=data_lm.train_ds.vocab, bs=16, text_cols = "ABSTRACT", label_cols = labels, valid_pct = 0.1)

In [ ]:
data_clas

In [ ]:
bs=16

In [ ]:
#torch.cuda.set_device(1)
#torch.cuda.set_device(0)

### Step 1. Training a Language Model

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

### Discrimiative Fine-Tuning

*learn.unfreeze()* makes all the layers of AWD_LSTM trainable. We can set a training rate using slice() function, which trains the last layer at 1e-02, while groups (of layers) in between would have geometrically reducing learning rates.

### Slated Triangular Learning Rates

This can achieved simply by using fit_one_cycle() method in fast.ai

### Gradual Unfreezing

Though I've not experimented with this here, the idea is pretty simple. In the start we keep the initial layers of model as un-trainable, and then we slowly unfreeze earlier layers, as we keep on training. I'll cover this in detail in next post

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

In [ ]:
learn.predict("This is a review about", n_words=10)

In [ ]:
learn.predict("This game is one of the ", n_words=10)

This performs pretty well in predicting the next few words of the review. Time to save this and try it on our classification task.

In [ ]:
learn.model_dir = Path('/kaggle/working/')

In [ ]:
learn.save(file = Path('language_model'))
learn.save_encoder(Path('language_model_encoder'))

### Step 2. Classification Task using Language Model as encoder

In [ ]:
F1_01 = MultiLabelFbeta(beta=1, average="micro", thresh = 0.1)
F1_03 = MultiLabelFbeta(beta=1, average="micro", thresh = 0.3)
F1_05 = MultiLabelFbeta(beta=1, average="micro", thresh = 0.5)
#learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, error_rate, F1_01, F1_03, F1_05]).to_fp16()
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, error_rate, F1_01, F1_03, F1_05])
learn.model_dir = Path('/kaggle/working/')
learn.load_encoder('language_model_encoder')

In [ ]:
data_clas.show_batch()

In [ ]:
#learn.freeze_to(-1)

In [ ]:
learn.fit_one_cycle(1, 5e-2)

In [ ]:
#learn.freeze_to(-2)

In [ ]:
#learn.fit_one_cycle(1, 5e-3)

In [ ]:
#learn.freeze_to(-3)

In [ ]:
#learn.fit_one_cycle(1, 5e-4)

In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data_clas.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

#test_preds = get_preds_as_nparray(DatasetType.Test)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4, 1e-3))

In [ ]:
preds1 = learn.get_preds(DatasetType.Test)[0].detach().cpu().numpy()
test_preds = preds1.copy()

thresh = 0.3
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_phase1_layer1_" + str(thresh) + ".csv", index=True)

thresh = 0.5
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_phase1_layer1_" + str(thresh) + ".csv", index=True)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-5, 1e-4))

In [ ]:
preds1 = learn.get_preds(DatasetType.Test)[0].detach().cpu().numpy()
test_preds = preds1.copy()

thresh = 0.3
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_phase1_layer2_" + str(thresh) + ".csv", index=True)

thresh = 0.5
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_phase1_layer2_" + str(thresh) + ".csv", index=True)

In [ ]:
#learn.lr_find()
#learn.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
#learn.unfreeze()
#learn.fit_one_cycle(4, slice(1e-6, 1e-5))

In [ ]:
#learn.lr_find()
#learn.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
learn.fit_one_cycle(2, slice(1e-6, 1e-5))

In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data_clas.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

#test_preds = get_preds_as_nparray(DatasetType.Test)

In [ ]:
preds1 = learn.get_preds(DatasetType.Test)[0].detach().cpu().numpy()
#preds2 = get_preds_as_nparray(DatasetType.Test)

In [ ]:
test_preds = preds1.copy()

In [ ]:
pd.DataFrame(test_preds).to_csv("/kaggle/working/output.csv")

In [ ]:
thresh = 0.3
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_layer2_" + str(thresh) + ".csv", index=True)

In [ ]:
thresh = 0.5
test_preds2 = test_preds.copy()
test_preds2[test_preds2<=thresh] = 0
test_preds2[test_preds2>thresh] = 1
x = pd.DataFrame(test_preds2, columns = labels)
x.index = test_dataset['ID']
x.to_csv("predictions_awd_lstm_new_layer2_" + str(thresh) + ".csv", index=True)